In [1]:
import pandas as pd
import numpy as np
import re

import time # importing time module

In [2]:
imdb_df=pd.read_csv('output/IMDB_movies.csv')
imdb_df['Year'] = pd.to_numeric(imdb_df['Year'], errors='coerce')
imdb_df.head()

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,"20,000 Leagues Under the Sea",1954.0,G,20 Jul 1955,127,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,019",tt0046672
1,3 Ninjas,1992.0,PG,07 Aug 1992,84,1 nomination,36.0,5.3,"21,478",tt0103596
2,A Civil Action,1998.0,PG-13,08 Jan 1999,115,Nominated for 2 Oscars. 5 wins & 10 nomination...,68.0,6.6,"28,628",tt0120633
3,A Kid in King Arthur's Court,1995.0,PG,11 Aug 1995,89,1 nomination,34.0,4.8,"6,143",tt0113538
4,A Low Down Dirty Shame,1994.0,R,23 Nov 1994,108,1 nomination,-1.0,5.9,"7,204",tt0110399


In [3]:
movielens_df=pd.read_csv('../resource/ml-25m/movies.csv')
movielens_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movielens_df['movie_title'] = movielens_df['title'].str.split(' \(\d', 1).str[0]
movielens_df['year'] = movielens_df['title'].str.extract('(\(\d{4}\)$)')
movielens_df['year'] = movielens_df['year'].str.replace('\)','')
movielens_df['year'] = movielens_df['year'].str.replace('\(','')
movielens_df.drop('title', axis=1, inplace=True)

In [5]:
movielens_df['year'] = pd.to_numeric(movielens_df['year'], errors='coerce')
movielens_df.head()

,movieId,genres,movie_title,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995.0
1,2,Adventure|Children|Fantasy,Jumanji,1995.0
2,3,Comedy|Romance,Grumpier Old Men,1995.0
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995.0
4,5,Comedy,Father of the Bride Part II,1995.0


In [6]:
movielens_df[movielens_df['year'].isnull()]

,movieId,genres,movie_title,year
8463,25936,Drama,"Babe Ruth Story, The",NaN
8602,26137,Action|Drama|War,"Heroes of Telemark, The",NaN
8910,26628,Crime|Horror|Mystery|Thriller,Jack's Back,NaN
9120,27189,Action|Drama,After the Rain (Ame agaru),NaN
9763,32497,Drama|Romance,Love Letter,NaN
...,...,...,...,...
62071,207714,(no genres listed),Tales of Found Footage,NaN
62104,207884,(no genres listed),Enduring Destiny,NaN
62285,208597,Documentary,Punk the Capital: Building a Sound Movement,NaN
62326,208763,(no genres listed),Yosemite: The Fate of Heaven,NaN


In [7]:
#Merge the Disney IMDB and MovieLens dataset on movie_title ONLY
merged_df =pd.merge(imdb_df, movielens_df, how='left', left_on=['Title','Year'], right_on=['movie_title','year'])
merged_df.head()

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,genres,movie_title,year
0,"20,000 Leagues Under the Sea",1954.0,G,20 Jul 1955,127,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,019",tt0046672,1019.0,Adventure|Drama|Sci-Fi,"20,000 Leagues Under the Sea",1954.0
1,3 Ninjas,1992.0,PG,07 Aug 1992,84,1 nomination,36.0,5.3,"21,478",tt0103596,4748.0,Action|Children|Comedy,3 Ninjas,1992.0
2,A Civil Action,1998.0,PG-13,08 Jan 1999,115,Nominated for 2 Oscars. 5 wins & 10 nomination...,68.0,6.6,"28,628",tt0120633,NaN,NaN,NaN,NaN
3,A Kid in King Arthur's Court,1995.0,PG,11 Aug 1995,89,1 nomination,34.0,4.8,"6,143",tt0113538,NaN,NaN,NaN,NaN
4,A Low Down Dirty Shame,1994.0,R,23 Nov 1994,108,1 nomination,-1.0,5.9,"7,204",tt0110399,NaN,NaN,NaN,NaN


In [10]:
#locate all the movies that are in IMDB & MovieLens
merged_df.loc[~merged_df['Title'].isna()]

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,genres,movie_title,year
0,"20,000 Leagues Under the Sea",1954.0,G,20 Jul 1955,127,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,019",tt0046672,1019.0,Adventure|Drama|Sci-Fi,"20,000 Leagues Under the Sea",1954.0
1,3 Ninjas,1992.0,PG,07 Aug 1992,84,1 nomination,36.0,5.3,"21,478",tt0103596,4748.0,Action|Children|Comedy,3 Ninjas,1992.0
2,A Civil Action,1998.0,PG-13,08 Jan 1999,115,Nominated for 2 Oscars. 5 wins & 10 nomination...,68.0,6.6,"28,628",tt0120633,NaN,NaN,NaN,NaN
3,A Kid in King Arthur's Court,1995.0,PG,11 Aug 1995,89,1 nomination,34.0,4.8,"6,143",tt0113538,NaN,NaN,NaN,NaN
4,A Low Down Dirty Shame,1994.0,R,23 Nov 1994,108,1 nomination,-1.0,5.9,"7,204",tt0110399,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,You Again,2010.0,PG,24 Sep 2010,105,2 wins,28.0,5.8,"51,344",tt1414382,80858.0,Comedy,You Again,2010.0
547,The Young Black Stallion,2003.0,G,25 Dec 2003,50,1 nomination,51.0,5.4,"1,163",tt0318850,NaN,NaN,NaN,NaN
548,Zokkomon,2011.0,PG,22 Apr 2011,109,NaN,-1.0,4.1,400,tt1605790,182339.0,Sci-Fi,Zokkomon,2011.0
549,Zootopia,2016.0,PG,04 Mar 2016,108,Won 1 Oscar. 47 wins & 72 nominations total,78.0,8.0,"477,250",tt2948356,152081.0,Action|Adventure|Animation|Children|Comedy,Zootopia,2016.0


In [38]:
#Is there any duplicates in MovieLens? We need the movieId
merged_df.loc[merged_df['Title'].duplicated()]

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,genres,movie_title,year
258,Aladdin,1992.0,G,25 Nov 1992,90,Won 2 Oscars. 32 wins & 22 nominations total,86.0,8.0,"395,426",tt0103639,114240.0,Adventure|Animation|Children|Comedy|Fantasy,Aladdin,1992.0
277,Casanova,2005.0,R,06 Jan 2006,112,5 wins & 3 nominations,57.0,6.5,"53,900",tt0402894,128862.0,Comedy|Drama|Romance,Casanova,2005.0


In [18]:
#Create a list of movie Title that are duplicated so we can resolved the MovieLens movieId
duplicate_title_list = merged_df['Title'].loc[merged_df['Title'].duplicated()].tolist()

In [39]:
#find all the duplicated titles from MovieLens
duplicates_df = merged_df.loc[merged_df['Title'].isin(duplicate_title_list)]
#duplicates_df.reset_index(drop=True, inplace=True)
duplicates_df

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,genres,movie_title,year
257,Aladdin,1992.0,G,25 Nov 1992,90,Won 2 Oscars. 32 wins & 22 nominations total,86.0,8.0,"395,426",tt0103639,588.0,Adventure|Animation|Children|Comedy|Musical,Aladdin,1992.0
258,Aladdin,1992.0,G,25 Nov 1992,90,Won 2 Oscars. 32 wins & 22 nominations total,86.0,8.0,"395,426",tt0103639,114240.0,Adventure|Animation|Children|Comedy|Fantasy,Aladdin,1992.0
276,Casanova,2005.0,R,06 Jan 2006,112,5 wins & 3 nominations,57.0,6.5,"53,900",tt0402894,42015.0,Action|Adventure|Comedy|Drama|Romance,Casanova,2005.0
277,Casanova,2005.0,R,06 Jan 2006,112,5 wins & 3 nominations,57.0,6.5,"53,900",tt0402894,128862.0,Comedy|Drama|Romance,Casanova,2005.0


In [40]:
#Copy the longest genres for the duplicates, so the duplicated ML MovieId will have the same genres

for i in merged_df.index:
    try:
         if merged_df["Title"][i] == merged_df["Title"][i+1] and len(merged_df['genres'][i]) > len(merged_df['genres'][i+1]):
            merged_df['genres'][i+1] = merged_df["genres"][i]
            print (merged_df["Title"][i])
            print (merged_df["Title"][i+1])
            print ('=======================')
    except:
        print("End")

Casanova
Casanova
End


/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
#check to see for the duplicated titles if the genres have been assigned correctly

merged_df.loc[merged_df['Title'].isin(duplicate_title_list)]

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID,movieId,genres,movie_title,year
257,Aladdin,1992.0,G,25 Nov 1992,90,Won 2 Oscars. 32 wins & 22 nominations total,86.0,8.0,"395,426",tt0103639,588.0,Adventure|Animation|Children|Comedy|Musical,Aladdin,1992.0
258,Aladdin,1992.0,G,25 Nov 1992,90,Won 2 Oscars. 32 wins & 22 nominations total,86.0,8.0,"395,426",tt0103639,114240.0,Adventure|Animation|Children|Comedy|Fantasy,Aladdin,1992.0
276,Casanova,2005.0,R,06 Jan 2006,112,5 wins & 3 nominations,57.0,6.5,"53,900",tt0402894,42015.0,Action|Adventure|Comedy|Drama|Romance,Casanova,2005.0
277,Casanova,2005.0,R,06 Jan 2006,112,5 wins & 3 nominations,57.0,6.5,"53,900",tt0402894,128862.0,Action|Adventure|Comedy|Drama|Romance,Casanova,2005.0


## Replace all MovieId from MovieLens to imdbID

### Do this for both movies & ratings dataset from MovieLens

In [ ]:
#For all movies & ratings, create a column imdbID if movieId matches merged_df
#Drop movieId